In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [3]:
pip install -U 'scikit-learn<0.24'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.8 MB 4.4 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.23.2 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.


In [4]:
!pip install sklearn-crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 965 kB 3.8 MB/s 


In [5]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from nltk.tokenize import word_tokenize

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [6]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [7]:
XTr=['Google wants a backend developer with good coding skills','There is a position open for machine learning engineer, Meta is looking for people with machine learning background']
XVal=['Amazon is looking for DevOps engineer']
org_b={'google':'C','amazon':'C','meta':'C','microsoft':'C','qualcomm':'C'}
skills_b={'coding':'S','machine':'S','learning':'S'}
jobs_b={'backend':'J','developer':'J','engineer':'J','manager':'J','devops':'J'}

def make(X,org_b,skills_b,jobs_b):
  big=[]
  for sentence in X:
    token=word_tokenize(sentence)
    small=[]
    token_t=nltk.pos_tag(token)
    for f,s in token_t:
      tag='O'
      if f.lower() in org_b:
        tag=org_b[f.lower()]
      if f.lower() in skills_b:
        tag=skills_b[f.lower()]
      if f.lower() in jobs_b:
        tag=jobs_b[f.lower()]
      small.append((f,s,tag))
    big.append(small)
  return big
XTr_=make(XTr,org_b,skills_b,jobs_b)
XVal_=make(XVal,org_b,skills_b,jobs_b)

In [8]:
def getfeatures(dat,i):
  word=dat[i][0]
  postag=dat[i][1]
  features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
  }
  if i>0:
    word=dat[i-1][0]
    postag=dat[i-1][1]
    features.update({
        '-1:word.lower()': word.lower(),    
        '-1:word.isupper()': word.isupper(),
        '-1:word.istitle()': word.istitle(),
        '-1:postag': postag,
        '-1:postag[:2]': postag[:2]
    })
  else:
    features['BOS']=True
  if i+1<len(dat):
    word=dat[i+1][0]
    postag=dat[i+1][1]
    features.update({
        '+1:word.lower()': word.lower(),    
        '+1:word.isupper()': word.isupper(),
        '+1:word.istitle()': word.istitle(),
        '+1:postag': postag,
        '+1:postag[:2]': postag[:2]
    })
  else:
    features['EOS']=True
  return features

In [9]:
def f2features(dat):
    return [getfeatures(dat, i) for i in range(len(dat))]

def f2labels(dat):
    return [label for token, postag, label in dat]

def f2tokens(dat):
    return [token for token, postag, label in dat]

In [10]:
XTr__=[f2features(x) for x in XTr_]
YTr__=[f2labels(x) for x in XTr_]

XVal__=[f2features(x) for x in XVal_]
YVal__=[f2labels(x) for x in XVal_]

In [11]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

In [12]:
crf.fit(XTr__, YTr__)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [13]:
labls=crf.classes_
labls.remove('O')

In [21]:
y_pred = crf.predict(XVal__)
metrics.flat_f1_score(YVal__, y_pred,average='weighted', labels=labls)

0.3333333333333333

In [23]:
print(metrics.flat_classification_report(YVal__, y_pred,digits=3))

              precision    recall  f1-score   support

           C      1.000     1.000     1.000         1
           J      0.000     0.000     0.000         2
           O      0.600     1.000     0.750         3

    accuracy                          0.667         6
   macro avg      0.533     0.667     0.583         6
weighted avg      0.467     0.667     0.542         6



In [24]:
y_pred

[['C', 'O', 'O', 'O', 'O', 'O']]